<a href="https://colab.research.google.com/github/arvindharul/simulation/blob/main/theater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://realpython.com/simpy-simulating-with-python/

In [1]:
import simpy
import random
import statistics
import numpy as np

The goal is to find the optimal number of employees that gives an average wait time of less than 10 minutes

**Cashiers** are a **resource** that the theater makes available to its customers, and they help moviegoers through the **process of purchasing a ticket**. 


Other Resources 
1. Ushers to check tickets (3 seconds)
2. Servers to sell food (1 to 5 minutes)

In [2]:

class Theater(object):
  def __init__(self,env,num_cashiers,num_servers,num_ushers):
    self.env = env
    self.cashier = simpy.Resource(env,num_cashiers)
    self.server = simpy.Resource(env,num_servers)
    self.usher = simpy.Resource(env,num_ushers)

  def purchase_ticket(self,moviegoer):
    #It takes roughtly 1 to 2 minutes for the cashier to issue a ticket
    yield self.env.timeout(random.randint(1,3))
  
  def check_ticket(self,moviegoer):
    yield self.env.timeout(3/60)
  
  def sell_food(self, moviegoer):
    yield self.env.timeout(random.randint(1,5))




Now you need a moviegoer to use them. When a moviegoer arrives at the theater, they’ll request a resource, wait for its process to complete, and then leave. You’ll create a function, called go_to_movies(), to keep track of this:

In [3]:
def go_to_movies(env,moviegoer,theater):
  arrival_time = env.now

  
  with theater.cashier.request() as request:
        #Requests and waits for cashier to be available
        yield request
        #When available initiate purchase process
        yield env.process(theater.purchase_ticket(moviegoer))
  
  with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))
  
  #optional step of buying food from the concession stand
  if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))
          
  
  # Moviegoer heads into the theater
  wait_times.append(env.now - arrival_time)

Now you’ll need to define a function to run the simulation. 

run_theater() will be responsible for creating an instance of a theater and generating moviegoers until the simulation stops. The first thing this function should do is create an instance of a theater:



In [4]:
# the simulation will generate 3 moviegoers to start and begin moving them through the theater with go_to_movies(). 
# After that, new moviegoers will arrive at the theater with an interval of 12 seconds and 
# move through the theater in their own time.

def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)

    #Intially we will have 3 people waiting when we open
    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))
    
    #On an average we might get a person every 20 seconds
    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))

In [5]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)

In [6]:
# This list will contain the total amount of time each
# moviegoer spends moving through the theater, from arrival to sitting in their seat.
wait_times = []
num_cashiers=11
num_servers=4
num_ushers=1
env = simpy.Environment()
env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
env.run(until=90)

print(np.mean(wait_times))


10.082703488372083
